# Extreme-Field QED + Gravitational Coupling Analysis

Comprehensive analysis notebook for experiment results.

**Capabilities:**
- Load and visualize HDF5 experiment results
- Compare strain and power across geometries
- Analyze κ-constraints for anomalous coupling
- Generate publication-quality figures
- Energy scaling and sensitivity analysis

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from pathlib import Path
import pandas as pd

# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Results directory
RESULTS_DIR = Path('../results')

## 1. Load Experiment Results

Load HDF5 files from completed experiments.

In [ ]:
def load_experiment_hdf5(filepath):
    """Load experiment results from HDF5 file."""
    results = {}
    
    with h5py.File(filepath, 'r') as f:
        # Metadata
        results['name'] = f.attrs['experiment_name']
        results['description'] = f.attrs['description']
        
        # Gravitational results
        if 'gravitational' in f:
            results['gravitational'] = {}
            for obs_key in f['gravitational'].keys():
                obs_data = {}
                for key in f[f'gravitational/{obs_key}'].attrs.keys():
                    obs_data[key] = f[f'gravitational/{obs_key}'].attrs[key]
                for key in f[f'gravitational/{obs_key}'].keys():
                    obs_data[key] = f[f'gravitational/{obs_key}/{key}'][:]
                results['gravitational'][obs_key] = obs_data
        
        # Anomalous coupling
        if 'anomalous' in f:
            results['anomalous'] = {}
            for ansatz in f['anomalous'].keys():
                ansatz_data = {'F_characteristic': f[f'anomalous/{ansatz}'].attrs['F_characteristic']}
                kappa_const = {}
                for key in f[f'anomalous/{ansatz}/kappa_constraints'].attrs.keys():
                    kappa_const[key] = f[f'anomalous/{ansatz}/kappa_constraints'].attrs[key]
                ansatz_data['kappa_constraints'] = kappa_const
                results['anomalous'][ansatz] = ansatz_data
    
    return results

# Example: load colliding pulses result
# exp1 = load_experiment_hdf5(RESULTS_DIR / 'colliding_pulses_qed.h5')

## 2. Strain Time Series Visualization

In [ ]:
def plot_strain_timeseries(results, R_key='R_1.0m', components=['xx', 'yy', 'zz']):
    """Plot gravitational strain time series."""
    if 'h_timeseries' not in results['gravitational'][R_key]:
        print(f"No time series data for {R_key}")
        return
    
    h_t = results['gravitational'][R_key]['h_timeseries']
    T = len(h_t)
    
    fig, axes = plt.subplots(len(components), 1, figsize=(12, 3*len(components)), sharex=True)
    if len(components) == 1:
        axes = [axes]
    
    comp_map = {'xx': (0,0), 'yy': (1,1), 'zz': (2,2), 'xy': (0,1), 'xz': (0,2), 'yz': (1,2)}
    
    for ax, comp in zip(axes, components):
        i, j = comp_map[comp]
        ax.plot(h_t[:, i, j], linewidth=0.8)
        ax.set_ylabel(f'$h_{{{comp}}}$')
        ax.grid(True, alpha=0.3)
        ax.axhline(0, color='k', linestyle='--', alpha=0.5)
    
    axes[-1].set_xlabel('Time Step')
    fig.suptitle(f'Gravitational Strain at {R_key}\\n{results["name"]}', fontsize=14)
    plt.tight_layout()
    return fig

# Example usage:
# fig = plot_strain_timeseries(exp1)

## 3. Radiated Power Analysis

In [ ]:
def plot_radiated_power(results, R_key='R_1.0m'):
    """Plot GW radiated power time series."""
    if 'P_timeseries' not in results['gravitational'][R_key]:
        print(f"No power time series for {R_key}")
        return
    
    P_t = results['gravitational'][R_key]['P_timeseries']
    
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(P_t, linewidth=1.0, color='darkred')
    ax.set_xlabel('Time Step')
    ax.set_ylabel('Radiated GW Power [W]')
    ax.set_yscale('log')
    ax.grid(True, alpha=0.3, which='both')
    ax.set_title(f'GW Power at {R_key}\\n{results["name"]}')
    
    # Add average line
    P_avg = np.mean(P_t)
    ax.axhline(P_avg, color='orange', linestyle='--', label=f'Average: {P_avg:.2e} W')
    ax.legend()
    
    plt.tight_layout()
    return fig

## 4. Multi-Experiment Comparison

In [ ]:
def compare_experiments(experiment_files, metric='h_rms', R_key='R_10.0m'):
    """Compare a metric across multiple experiments."""
    data = []
    
    for filepath in experiment_files:
        results = load_experiment_hdf5(filepath)
        value = results['gravitational'][R_key][metric]
        data.append({
            'Experiment': results['name'],
            metric: value
        })
    
    df = pd.DataFrame(data)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(range(len(df)), df[metric], tick_label=df['Experiment'])
    ax.set_ylabel(metric)
    ax.set_yscale('log')
    ax.set_title(f'Comparison: {metric} at {R_key}')
    ax.grid(True, alpha=0.3, axis='y')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    return fig, df

# Example:
# files = [RESULTS_DIR / f for f in ['colliding_pulses_qed.h5', 'cavity_photon_reservoir.h5']]
# fig, df = compare_experiments(files)

## 5. Anomalous Coupling κ-Constraints

In [ ]:
def plot_kappa_constraints(results, ansatz_name='vector_potential_squared'):
    """Visualize κ required to reach each detection threshold."""
    if ansatz_name not in results['anomalous']:
        print(f"Ansatz {ansatz_name} not found")
        return
    
    constraints = results['anomalous'][ansatz_name]['kappa_constraints']
    
    detectors = list(constraints.keys())
    kappas = list(constraints.values())
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.barh(detectors, kappas, color='steelblue', edgecolor='black')
    
    ax.set_xlabel(r'Required $\kappa$ for Detection')
    ax.set_xscale('log')
    ax.set_title(f'Anomalous Coupling Constraints\\n{ansatz_name}\\n{results["name"]}')
    ax.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for bar, val in zip(bars, kappas):
        if val > 0:
            ax.text(val, bar.get_y() + bar.get_height()/2, 
                   f' {val:.2e}', va='center', fontsize=9)
    
    plt.tight_layout()
    return fig

# Example:
# fig = plot_kappa_constraints(exp1, 'field_invariant_F2')

## 6. Energy Scaling Analysis

In [ ]:
def energy_scaling_plot(sweep_results, param_name='E0'):
    """Plot how h_rms and P_avg scale with energy parameter."""
    # Assumes sweep_results is a list of (param_value, results) tuples
    
    params = []
    h_rms_vals = []
    P_avg_vals = []
    
    for param_val, results in sweep_results:
        params.append(param_val)
        h_rms_vals.append(results['gravitational']['R_10.0m']['h_rms'])
        P_avg_vals.append(results['gravitational']['R_10.0m']['P_avg'])
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # h_rms scaling
    ax1.loglog(params, h_rms_vals, 'o-', linewidth=2, markersize=8)
    ax1.set_xlabel(param_name)
    ax1.set_ylabel(r'$h_{\\rm rms}$')
    ax1.set_title('Strain Scaling')
    ax1.grid(True, alpha=0.3, which='both')
    
    # Fit power law
    log_params = np.log10(params)
    log_h = np.log10(h_rms_vals)
    slope_h, intercept_h = np.polyfit(log_params, log_h, 1)
    ax1.text(0.05, 0.95, f'Slope: {slope_h:.2f}', transform=ax1.transAxes, 
             va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # P_GW scaling
    ax2.loglog(params, P_avg_vals, 's-', linewidth=2, markersize=8, color='darkred')
    ax2.set_xlabel(param_name)
    ax2.set_ylabel(r'$P_{\\rm GW}$ [W]')
    ax2.set_title('Radiated Power Scaling')
    ax2.grid(True, alpha=0.3, which='both')
    
    log_P = np.log10(P_avg_vals)
    slope_P, intercept_P = np.polyfit(log_params, log_P, 1)
    ax2.text(0.05, 0.95, f'Slope: {slope_P:.2f}', transform=ax2.transAxes,
             va='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    return fig

## 7. Publication Figure: Detectability Map

In [ ]:
def publication_detectability_map(results, R_values=[1, 10, 100, 1000]):
    """Create publication-quality figure showing detectability across detectors and distances."""
    
    detector_thresholds = {
        'LIGO': 1e-21,
        'aLIGO': 1e-22,
        'ET': 1e-23,
        'Quantum (Asp.)': 1e-30,
    }
    
    # Compute h at each distance
    h_ref = results['gravitational']['R_1.0m']['h_rms']
    R_ref = 1.0
    
    fig, ax = plt.subplots(figsize=(10, 7))
    
    # Plot detector thresholds as horizontal lines
    colors = ['red', 'orange', 'green', 'blue']
    for (name, threshold), color in zip(detector_thresholds.items(), colors):
        ax.axhline(threshold, color=color, linestyle='--', linewidth=2, label=name, alpha=0.7)
    
    # Plot h(R) scaling
    R_array = np.logspace(0, 4, 100)
    h_array = h_ref * (R_ref / R_array)
    
    ax.loglog(R_array, h_array, 'k-', linewidth=3, label='Signal (1/R scaling)')
    
    ax.set_xlabel('Distance [m]', fontsize=14)
    ax.set_ylabel(r'Strain $h_{\\rm rms}$', fontsize=14)
    ax.set_title(f'Detectability Map\\n{results["name"]}', fontsize=16)
    ax.legend(fontsize=11, loc='best')
    ax.grid(True, alpha=0.3, which='both')
    
    # Shade detectable regions
    for (name, threshold), color in zip(detector_thresholds.items(), colors):
        # Find where h(R) > threshold
        R_detect = R_ref * (h_ref / threshold)
        if R_detect < R_array[-1]:
            ax.axvspan(R_array[0], min(R_detect, R_array[-1]), 
                      alpha=0.1, color=color)
    
    plt.tight_layout()
    return fig

## 8. Example Analysis Workflow

In [ ]:
# Complete analysis example (uncomment when HDF5 files exist)

# # Load experiment
# exp = load_experiment_hdf5(RESULTS_DIR / 'colliding_pulses_qed.h5')
# 
# # Plot strain
# fig_strain = plot_strain_timeseries(exp, R_key='R_10.0m')
# fig_strain.savefig('figures/strain_timeseries.png', dpi=300, bbox_inches='tight')
# 
# # Plot power
# fig_power = plot_radiated_power(exp, R_key='R_10.0m')
# fig_power.savefig('figures/radiated_power.png', dpi=300, bbox_inches='tight')
# 
# # Kappa constraints
# fig_kappa = plot_kappa_constraints(exp, 'vector_potential_squared')
# fig_kappa.savefig('figures/kappa_constraints.png', dpi=300, bbox_inches='tight')
# 
# # Detectability map
# fig_detect = publication_detectability_map(exp)
# fig_detect.savefig('figures/detectability_map.png', dpi=300, bbox_inches='tight')
# 
# print("Analysis complete!")

## 9. Summary Statistics Table

In [ ]:
def generate_summary_table(results_list):
    """Generate summary table for multiple experiments."""
    summary = []
    
    for results in results_list:
        R_key = 'R_10.0m'  # Standard comparison distance
        grav = results['gravitational'][R_key]
        
        row = {
            'Experiment': results['name'],
            'h_rms': f"{grav['h_rms']:.2e}",
            'h_max': f"{grav['h_max']:.2e}",
            'P_avg [W]': f"{grav['P_avg']:.2e}",
            'Peak Freq [Hz]': f"{grav.get('frequency_spectrum_peak_freq_Hz', 0):.2e}",
        }
        
        summary.append(row)
    
    df = pd.DataFrame(summary)
    return df

# Example:
# exps = [exp1, exp2, exp3]
# table = generate_summary_table(exps)
# print(table.to_markdown(index=False))  # For reports